In [16]:
# Load model directly
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm.auto import tqdm
from datasets import load_dataset

import transformers
from tqdm.auto import tqdm, trange
assert torch.cuda.is_available()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [17]:
%reload_ext autoreload
%autoreload 2

import sys
import os.path as osp

SRC_SUBDIR = '../src/'
SRC_SUBDIR = osp.abspath(SRC_SUBDIR)
if SRC_SUBDIR not in sys.path:
    print(f'Adding source directory to the sys.path: {SRC_SUBDIR!r}')
    sys.path.insert(1, SRC_SUBDIR)

In [18]:
!ls ../src

data  features	__init__.py  models  __pycache__  utils  visualization


In [27]:
sys.path

['/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python310.zip',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python310.zip/../',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python310.zip/../',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python310.zip/../src',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python310.zip/../',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python310.zip/../',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python310.zip/../',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python310.zip/../src',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python310.zip/../',
 '/home/usoltsev/study/repositories/rugpt-memory/src',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python3.10',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python3.10/lib-dynload',
 '',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python3.10/site-packages',
 '/tmp/tmpq3ceq4ua']

In [28]:
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '../../'))
sys.path

['/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python310.zip',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python310.zip/../../',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python310.zip/../',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python310.zip/../',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python310.zip/../src',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python310.zip/../',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python310.zip/../',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python310.zip/../',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python310.zip/../src',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python310.zip/../',
 '/home/usoltsev/study/repositories/rugpt-memory/src',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python3.10',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python3.10/lib-dynload',
 '',
 '/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python3.10/site-packages',
 '/tmp/tmpq3ceq4ua']

In [32]:
pwd

'/home/usoltsev/study/repositories/rugpt-memory/notebooks'

In [33]:
sys.path.insert(1, '/home/usoltsev/study/repositories/rugpt-memory/')

In [34]:
from src.models.ltm_gpt.ltm_gpt import LTM_GPT
from src.utils.train_config import load_config
from src.models.load_base_model import load_base_model

In [ ]:
!tree -L 1 ..

In [35]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import os
os.environ['CURL_CA_BUNDLE'] = ''

In [36]:
main_config = load_config('/home/usoltsev/study/repositories/rugpt-memory/configs/finetuning_codeparrot.yml')

model, tokenizer = load_base_model(main_config)

[2024-03-23 17:50:17,532: INFO] Loading base model (load_base_model.py:9)


/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python3.10/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Loading checkpoint shards:   0%|                                                   | 0/6 [00:00<?, ?it/s]/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|███████████████████████████████████████████| 6/6 [00:07<00:00,  1.29s/it]
/home/usoltsev/miniforge3/envs/r

In [38]:
!nvidia-smi

Sat Mar 23 17:50:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.146.02             Driver Version: 535.146.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:19:00.0 Off |                  N/A |
| 30%   33C    P5             104W / 350W |  12865MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [39]:
cnt_blocks_with_memory = 2

model = LTM_GPT(
    model,
    cnt_blocks_with_memory=cnt_blocks_with_memory
)

In [40]:
!nvidia-smi

Sat Mar 23 17:53:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.146.02             Driver Version: 535.146.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:19:00.0 Off |                  N/A |
| 30%   31C    P2             125W / 350W |  16485MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [41]:
!pwd

/home/usoltsev/study/repositories/rugpt-memory/notebooks


In [43]:
from accelerate import Accelerator

accelerator = Accelerator()

In [44]:
model_0 = accelerator.prepare(model)

In [45]:
!nvidia-smi

Sat Mar 23 18:13:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.146.02             Driver Version: 535.146.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:19:00.0 Off |                  N/A |
| 30%   25C    P8              27W / 350W |  16485MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [46]:
model = model_0

In [47]:
for param in model.transformer.h[-cnt_blocks_with_memory:].parameters():
    param.requires_grad=True
    # param.data = param.data.to(torch.float32)

for param in model.transformer.ln_f.parameters():
    param.requires_grad=True
    # param.data = param.data.to(torch.float32)

for param in model.lm_head.parameters():
    param.requires_grad=True
    # param.data = param.data.to(torch.float32)

prompts = ['import', 'from', 'while', 'try', 'if', 'for',
           'torch']  # feel free to add a few more that are not 100% assiciated with Python

MAX_STEPS = 100

def custom_generate(prompt, model, device, max_steps):
    batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
    print(batch)

    for i in range(max_steps):
        outputs = model(**batch)
        # print(outputs)
        probs = outputs.logits[0, -1].nan_to_num(nan=0.0).div(0.8).softmax(-1)  # .argmax(-1).reshape(1, 1)
        old_token = outputs.logits[0, -1].argmax(-1).reshape(1, 1)
        # print(old_token)
        next_token = torch.multinomial(probs, 1).reshape(1, 1)
        # print(next_token)
        batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
        batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)
        break

    return tokenizer.decode(batch['input_ids'][0].cpu().numpy().tolist()[1:])

device = torch.device("cuda")

after_finetuning_samples = []
for prompt in prompts:
    after_finetuning_samples.append(custom_generate(prompt, model, device, MAX_STEPS))
print(after_finetuning_samples)

{'input_ids': tensor([[33076]], device='cuda:0'), 'attention_mask': tensor([[1]], device='cuda:0')}


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:1 and cuda:0! (when checking argument for argument weight in method wrapper_CUDA__native_layer_norm)

In [50]:
from src.data.load_codeparrot_dataset import load_codeparrot_dataset
from torch.utils.tensorboard import SummaryWriter
dataset = load_codeparrot_dataset(tokenizer)

# logger.info('Initializing tensorboard')
tensorboard_logs_dir = f'logs/tensorboard/{main_config.exp_name}/'
tensorboard_writer = SummaryWriter(tensorboard_logs_dir)

[2024-03-23 18:26:34,977: INFO] Loading dataset (load_codeparrot_dataset.py:7)


/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python3.10/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python3.10/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python3.10/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.w

In [51]:
# logger.info('Training')
model._hf_peft_config_loaded = True # silence warnings - for research it is ok
model.config.use_cache = False # silence warnings from torch

trainer_args = main_config.trainer_args

In [57]:
accelerator.device

device(type='cuda')

In [ ]:
accelerator = Accelerator()

In [53]:
from transformers.integrations import TensorBoardCallback
trainer = accelerator.prepare(transformers.Trainer(
    model=model, train_dataset=dataset['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=trainer_args.per_device_train_batch_size,
        gradient_accumulation_steps=trainer_args.gradient_accumulation_steps,
        warmup_steps=trainer_args.warmup_steps,
        max_steps=trainer_args.max_steps,
        learning_rate=trainer_args.learning_rate,
        fp16=trainer_args.fp16,
        logging_steps=trainer_args.logging_steps,
        output_dir=f'checkpoints/{main_config.exp_name}/outputs',
        report_to=trainer_args.report_to,
        gradient_checkpointing=True,
        gradient_checkpointing_kwargs={'use_reentrant':True}
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    callbacks=[TensorBoardCallback(tb_writer=tensorboard_writer)]
))

/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 200.00 MiB. GPU 0 has a total capacty of 23.69 GiB of which 53.94 MiB is free. Including non-PyTorch memory, this process has 23.62 GiB memory in use. Of the allocated memory 23.30 GiB is allocated by PyTorch, and 13.94 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.train()



In [ ]:
model.config.use_cache = True

In [2]:
model = AutoModelForCausalLM.from_pretrained(
    "ai-forever/ruGPT-3.5-13B",
    torch_dtype=torch.float16,
    device_map='auto',
    low_cpu_mem_usage=True,
    offload_state_dict=True, 
    cache_dir="/home/usoltsev/study/repositories/rugpt-memory/checkpoints/base/huggingface/"
)

Loading checkpoint shards:   0%|                                                   | 0/6 [00:00<?, ?it/s]/home/usoltsev/miniforge3/envs/rugpt_dev/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|███████████████████████████████████████████| 6/6 [00:15<00:00,  2.65s/it]


In [4]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50272, 5120)
    (wpe): Embedding(2048, 5120)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-39): 40 x GPT2Block(
        (ln_1): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=5120, out_features=50272, bias=False)
)

In [11]:
model.device

device(type='cuda', index=0)

In [10]:
model.transformer.h[0].device

AttributeError: 'GPT2Block' object has no attribute 'device'

In [3]:
# optimization 
model.gradient_checkpointing_enable() # memory optimization: only store a small subset of activations, re-compute the rest.
model.enable_input_require_grads() # override an implementation quirk in gradient checkpoints that disables backprop unless inputs require grad
class CastOutputToFloat(torch.nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head) # cast model ouputs to unfuct the top-k sampler
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50272, 5120)
    (wpe): Embedding(2048, 5120)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-39): 40 x GPT2Block(
        (ln_1): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): CastOutputToFloat(
    (0): Linear(in_features=5120, out_features=50272, bias=False)
  )
)

In [4]:
class DenseNetwork(nn.Module):
    """ DenseNetwork layer(FeedForward in original paper) """
    def __init__(
        self, 
        embed_dim=5120,
        hidden_size=10240, 
        dtype=torch.float32,
        initialize_with_zeros=False
    ):
        super().__init__()
        
        self.embed_dim = embed_dim
        self.hidden_size = hidden_size
        self.dtype = dtype
        
        self.ln1 = nn.Linear(self.embed_dim, self.hidden_size, dtype=self.dtype)
        self.relu = nn.ReLU()
        self.ln2 = nn.Linear(self.hidden_size, self.embed_dim, dtype=self.dtype)
        
        if initialize_with_zeros:
            nn.init.zeros_(self.ln1.weight)
            nn.init.zeros_(self.ln1.bias)
            nn.init.zeros_(self.ln2.weight)
            nn.init.zeros_(self.ln2.bias)
    
    def forward(self, x): # x: (sentence_length, batch_size, self.embed_dim)
        x = self.ln1(x)
        x = self.relu(x)
        x = self.ln2(x)
        return x

In [5]:
class LTMGPT2Block(nn.Module):
    """ Custom LTMGPT2Block layer with memory """
    def __init__(
        self, 
        gpt2_block,
        num_heads=4,
        attn_dropout=0.1,
        dense_network_hidden_size=10240,
        dtype=torch.float32
    ):
        super().__init__()
        self.gpt2_block = gpt2_block
        
        self.embed_dim = self.gpt2_block.ln_1.normalized_shape[0]
        self.dense_network_hidden_size = dense_network_hidden_size
        
        assert dtype in [torch.float16, torch.float32]
        
        # self.memory: ( , , ) / (target_sentence_length, batch_size, self.embed_dim) (5120) | torch.FloatTensor / nn.Embedding
        self.memory = None
        
        # goal: convert memory from ( , , ) to (source_sentence_length, batch_size, self.embed_dim)
        self.dense_network1 = DenseNetwork(
            embed_dim=self.embed_dim,
            hidden_size=self.dense_network_hidden_size, 
            dtype=dtype,
            initialize_with_zeros=False
        )
        
        self.attn = nn.MultiheadAttention( # TODO masked ????
            embed_dim=self.embed_dim, 
            num_heads=num_heads, 
            dropout=attn_dropout,
            batch_first=False,
            dtype=dtype
        )
        
        self.ln1 = nn.LayerNorm(self.embed_dim)
        
        self.dense_network2 = DenseNetwork(
            embed_dim=self.embed_dim,
            hidden_size=self.dense_network_hidden_size, 
            dtype=dtype,
            initialize_with_zeros=True
        )
        
        self.ln2 = nn.LayerNorm(self.embed_dim)
        
        
        
    
    def forward(self, x): # x: (sentence_length, batch_size, self.embed_dim)
        assert not self.memory
        
        # TransformerBlock
        query = self.gpt2_block(x) # query: (sentence_length, batch_size, self.embed_dim)
        residual = query
        
        # DenseNetowork
        memory = self.dense_network1(self.memory)
        
        # MultiHead Attention
        key, value = memory, memory
        x, _ = self.attn(
            query=query, 
            key=key, 
            value=value
        )
        
        # Norm & Concat
        x = x + residual
        if self.dtype == torch.float16:
            x = self.ln1(x.float()).type(torch.float16)
        else:
            x = self.ln1(x)
        
        # DenseNetowork initialized with zeroes
        x = self.dense_network2(x)
        
        # Norm & Concat
        x = x + residual
        if self.dtype == torch.float16:
            x = self.ln2(x.float()).type(torch.float16)
        else:
            x = self.ln2(x)
        
        return x
    
    def update_memory(new_memory):
        self.memory = new_memory

In [6]:
from transformers.models.gpt2.modeling_gpt2 import GPT2Model

In [7]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50272, 5120)
    (wpe): Embedding(2048, 5120)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-39): 40 x GPT2Block(
        (ln_1): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): CastOutputToFloat(
    (0): Linear(in_features=5120, out_features=50272, bias=False)
  )
)

In [8]:
model.transformer#.output_hidden_states

GPT2Model(
  (wte): Embedding(50272, 5120)
  (wpe): Embedding(2048, 5120)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-39): 40 x GPT2Block(
      (ln_1): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
)

In [9]:
model.transformer.config.output_hidden_states

False

In [10]:
model.transformer.config.output_hidden_states = True

In [11]:
model.transformer.config.output_hidden_states

True

In [12]:
model.model_parallel

False

In [13]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50272, 5120)
    (wpe): Embedding(2048, 5120)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-39): 40 x GPT2Block(
        (ln_1): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): CastOutputToFloat(
    (0): Linear(in_features=5120, out_features=50272, bias=False)
  )
)

In [14]:
from transformers.models.gpt2.modeling_gpt2 import GPT2LMHeadModel
from transformers.utils import (
    ModelOutput,
    add_code_sample_docstrings,
    add_start_docstrings,
    add_start_docstrings_to_model_forward,
    logging,
    replace_return_docstrings,
)
from transformers.modeling_outputs import (
    BaseModelOutputWithPastAndCrossAttentions,
    CausalLMOutputWithCrossAttentions,
    QuestionAnsweringModelOutput,
    SequenceClassifierOutputWithPast,
    TokenClassifierOutput,
)
from typing import Optional, Tuple, Union

In [15]:
class LTM_GPT(GPT2LMHeadModel):
    """ Custom LTM GPT2 layer with memory """
    def __init__(self, model_: GPT2LMHeadModel, cnt_blocks_with_memory=2):
        super().__init__(model.config)
        print(type(model_))
        self.base_model = model_
        print(type(self.base_model))
        self.transformer = model.transformer
        self.transformer.h = model_.transformer.h[:-cnt_blocks_with_memory]
        
        self.transformer_ltm_blocks = nn.ModuleList([
            LTMGPT2Block(model_.transformer.h[-cnt_blocks_with_memory+i]) for i in range(cnt_blocks_with_memory)
        ])
        
        self.lm_head = model_.lm_head

        # Model parallel
        # self.model_parallel = False
        # self.device_map = None

        # Initialize weights and apply final processing
        # self.post_init()
    
    # @add_start_docstrings_to_model_forward(GPT2_INPUTS_DOCSTRING)
    # @add_code_sample_docstrings(
    #     checkpoint=_CHECKPOINT_FOR_DOC,
    #     output_type=CausalLMOutputWithCrossAttentions,
    #     config_class=_CONFIG_FOR_DOC,
    # )
    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[Tuple[Tuple[torch.Tensor]]] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.Tensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, CausalLMOutputWithCrossAttentions]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for language modeling. Note that the labels **are shifted** inside the model, i.e. you can set
            `labels = input_ids` Indices are selected in `[-100, 0, ..., config.vocab_size]` All labels set to `-100`
            are ignored (masked), the loss is only computed for labels in `[0, ..., config.vocab_size]`
        """
        return_dict = return_dict if return_dict is not None else self.base_model.config.use_return_dict

        transformer_outputs = self.transformer(
            input_ids,
            past_key_values=past_key_values,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_attention_mask,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=True,
            return_dict=return_dict,
        )
        hidden_states = transformer_outputs[0]
        
        # Init memory as hidden_states from 37 layers
        # BaseModelOutputWithPastAndCrossAttentions(
        #     last_hidden_state=hidden_states,
        #     past_key_values=presents,
        #     hidden_states=all_hidden_states,
        #     attentions=all_self_attentions,
        #     cross_attentions=all_cross_attentions,
        # )    
        
        print(len(transformer_outputs))
        memory = transformer_outputs[2][37]
        
        for block in self.transformer_ltm_blocks:
            block.update_memory(memory)
            hidden_states = block(hidden_states)

        # Set device for model parallelism
        if self.base_model.model_parallel:
            torch.cuda.set_device(self.transformer.first_device)
            hidden_states = hidden_states.to(self.lm_head.weight.device)

        lm_logits = self.lm_head(hidden_states)

        loss = None
        if labels is not None:
            # move labels to correct device to enable model parallelism
            labels = labels.to(lm_logits.device)
            # Shift so that tokens < n predict n
            shift_logits = lm_logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # Flatten the tokens
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

        if not return_dict:
            output = (lm_logits,) + transformer_outputs[1:]
            return ((loss,) + output) if loss is not None else output

        return CausalLMOutputWithCrossAttentions(
            loss=loss,
            logits=lm_logits,
            past_key_values=transformer_outputs.past_key_values,
            hidden_states=transformer_outputs.hidden_states,
            attentions=transformer_outputs.attentions,
            cross_attentions=transformer_outputs.cross_attentions,
        )

In [1]:
!pwd

/home/usoltsev/study/repositories/rugpt-memory/notebooks


In [17]:
model.transformer

GPT2Model(
  (wte): Embedding(50272, 5120)
  (wpe): Embedding(2048, 5120)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-39): 40 x GPT2Block(
      (ln_1): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
)

In [18]:
type(model)

transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel

In [19]:
type(model.transformer)

transformers.models.gpt2.modeling_gpt2.GPT2Model

In [20]:
model = LTM_GPT(model)

<class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>
<class 'transformers.models.gpt2.modeling_gpt2.GPT2Model'>


In [21]:
model

LTM_GPT(
  (transformer): GPT2Model(
    (wte): Embedding(50272, 5120)
    (wpe): Embedding(2048, 5120)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-37): 38 x GPT2Block(
        (ln_1): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): CastOutputToFloat(
    (0): Linear(in_features=5120, out_features=50272, bias=False)
  )
  (base_model): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embeddin

In [22]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruGPT-3.5-13B")

In [23]:
code_dataset = load_dataset("codeparrot/codeparrot-clean-valid")

Repo card metadata block was not found. Setting CardData to empty.


In [24]:
prompts =  ['import', 'from', 'while', 'try', 'if', 'for', 'torch']  # feel free to add a few more that are not 100% assiciated with Python

MAX_STEPS = 100

for prompt in tqdm(prompts):
    print(tokenizer(prompt, return_tensors='pt', return_token_type_ids=False))

100%|██████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 420.63it/s]

{'input_ids': tensor([[33076]]), 'attention_mask': tensor([[1]])}
{'input_ids': tensor([[34958]]), 'attention_mask': tensor([[1]])}
{'input_ids': tensor([[29631]]), 'attention_mask': tensor([[1]])}
{'input_ids': tensor([[  89, 2286]]), 'attention_mask': tensor([[1, 1]])}
{'input_ids': tensor([[1271]]), 'attention_mask': tensor([[1]])}
{'input_ids': tensor([[9949]]), 'attention_mask': tensor([[1]])}
{'input_ids': tensor([[23652,  1028]]), 'attention_mask': tensor([[1, 1]])}


In [25]:
def custom_generate(prompt, model, device, max_steps):
    batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
    print(batch)

    for i in range(max_steps):
        outputs = model(**batch)
        #print(outputs)
        probs = outputs.logits[0, -1].nan_to_num(nan=0.0).div(0.8).softmax(-1) #.argmax(-1).reshape(1, 1)
        old_token = outputs.logits[0, -1].argmax(-1).reshape(1, 1)
        #print(old_token)
        next_token = torch.multinomial(probs, 1).reshape(1, 1)
        #print(next_token)
        batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
        batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)
        break

    return tokenizer.decode(batch['input_ids'][0].cpu().numpy().tolist()[1:])

In [26]:
after_finetuning_samples = []
for prompt in tqdm(prompts):
    after_finetuning_samples.append(custom_generate(prompt, model, device, MAX_STEPS))
after_finetuning_samples

  0%|                                                                                           | 0/7 [00:00<?, ?it/s]

{'input_ids': tensor([[33076]], device='cuda:0'), 'attention_mask': tensor([[1]], device='cuda:0')}


  0%|                                                                                           | 0/7 [00:01<?, ?it/s]

BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[-0.1104, -0.3472, -0.0307,  ...,  0.0503, -0.2764,  0.0389]]],
       device='cuda:1', dtype=torch.float16, grad_fn=<ViewBackward0>), past_key_values=((tensor([[[[ 0.5664,  0.5801, -1.3389,  ...,  0.3323, -0.7427, -0.5698]],

         [[ 0.4031, -0.1638,  0.8931,  ..., -0.8618, -0.4988, -0.8623]],

         [[ 1.6885, -0.5928,  0.3030,  ..., -0.1799,  0.4023, -0.7407]],

         ...,

         [[-0.2412, -2.2031, -0.1914,  ...,  0.4146, -0.1887, -1.0791]],

         [[ 1.4297, -0.2235, -1.0244,  ..., -1.3994, -0.3193,  0.0953]],

         [[ 2.2207, -0.0444,  1.5391,  ..., -0.1519, -0.6484, -2.9902]]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<PermuteBackward0>), tensor([[[[ 0.5215,  0.1376, -0.3691,  ..., -0.3887,  0.5425,  0.0561]],

         [[ 0.4697,  0.1948,  0.1681,  ...,  0.1853, -0.4612, -0.4224]],

         [[ 0.1459,  0.1271, -0.0180,  ...,  0.0782, -0.1848,  0.1338]],

         ...,

       

TypeError: update_memory() takes 1 positional argument but 2 were given

In [ ]:
[-3 + i for i in range(3)]

In [ ]:
from transformers.utils import (
    ModelOutput,
    add_code_sample_docstrings,
    add_start_docstrings,
    add_start_docstrings_to_model_forward,
    logging,
    replace_return_docstrings,
)
from transformers.modeling_outputs import (
    BaseModelOutputWithPastAndCrossAttentions,
    CausalLMOutputWithCrossAttentions,
    QuestionAnsweringModelOutput,
    SequenceClassifierOutputWithPast,
    TokenClassifierOutput,
)
from typing import Optional, Tuple, Union

In [ ]:
GPT2_INPUTS_DOCSTRING = r"""
    Args:
        input_ids (`torch.LongTensor` of shape `(batch_size, input_ids_length)`):
            `input_ids_length` = `sequence_length` if `past_key_values` is `None` else
            `past_key_values[0][0].shape[-2]` (`sequence_length` of input past key value states). Indices of input
            sequence tokens in the vocabulary.

            If `past_key_values` is used, only `input_ids` that do not have their past calculated should be passed as
            `input_ids`.

            Indices can be obtained using [`AutoTokenizer`]. See [`PreTrainedTokenizer.encode`] and
            [`PreTrainedTokenizer.__call__`] for details.

            [What are input IDs?](../glossary#input-ids)
        past_key_values (`Tuple[Tuple[torch.Tensor]]` of length `config.n_layers`):
            Contains precomputed hidden-states (key and values in the attention blocks) as computed by the model (see
            `past_key_values` output below). Can be used to speed up sequential decoding. The `input_ids` which have
            their past given to this model should not be passed as `input_ids` as they have already been computed.
        attention_mask (`torch.FloatTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Mask to avoid performing attention on padding token indices. Mask values selected in `[0, 1]`:

            - 1 for tokens that are **not masked**,
            - 0 for tokens that are **masked**.

            If `past_key_values` is used, `attention_mask` needs to contain the masking strategy that was used for
            `past_key_values`. In other words, the `attention_mask` always has to have the length:
            `len(past_key_values) + len(input_ids)`

            [What are attention masks?](../glossary#attention-mask)
        token_type_ids (`torch.LongTensor` of shape `(batch_size, input_ids_length)`, *optional*):
            Segment token indices to indicate first and second portions of the inputs. Indices are selected in `[0,
            1]`:

            - 0 corresponds to a *sentence A* token,
            - 1 corresponds to a *sentence B* token.

            [What are token type IDs?](../glossary#token-type-ids)
        position_ids (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Indices of positions of each input sequence tokens in the position embeddings. Selected in the range `[0,
            config.max_position_embeddings - 1]`.

            [What are position IDs?](../glossary#position-ids)
        head_mask (`torch.FloatTensor` of shape `(num_heads,)` or `(num_layers, num_heads)`, *optional*):
            Mask to nullify selected heads of the self-attention modules. Mask values selected in `[0, 1]`:

            - 1 indicates the head is **not masked**,
            - 0 indicates the head is **masked**.

        inputs_embeds (`torch.FloatTensor` of shape `(batch_size, sequence_length, hidden_size)`, *optional*):
            Optionally, instead of passing `input_ids` you can choose to directly pass an embedded representation. This
            is useful if you want more control over how to convert `input_ids` indices into associated vectors than the
            model's internal embedding lookup matrix.

            If `past_key_values` is used, optionally only the last `inputs_embeds` have to be input (see
            `past_key_values`).
        use_cache (`bool`, *optional*):
            If set to `True`, `past_key_values` key value states are returned and can be used to speed up decoding (see
            `past_key_values`).
        output_attentions (`bool`, *optional*):
            Whether or not to return the attentions tensors of all attention layers. See `attentions` under returned
            tensors for more detail.
        output_hidden_states (`bool`, *optional*):
            Whether or not to return the hidden states of all layers. See `hidden_states` under returned tensors for
            more detail.
        return_dict (`bool`, *optional*):
            Whether or not to return a [`~utils.ModelOutput`] instead of a plain tuple.
"""
_CHECKPOINT_FOR_DOC = "openai-community/gpt2"
_CONFIG_FOR_DOC = "GPT2Config"

In [ ]:
class LTM_GPT2Model(GPT2Model):
    """ Custom LTM GPT2 layer with memory """
    def __init__(self, model: GPT2Model, cnt_blocks_with_memory=2):
        super().__init__(model.config)
        self.base_model = model
        
        self.embed_dim = self.base_model.embed_dim

        self.wte = self.base_model.wte
        self.wpe = self.base_model.wpe

        self.drop = self.base_model.drop
        self.h = self.base_model.h[:-cnt_blocks_with_memory]
        self.transformer_ltm_blocks_h = nn.ModuleList([
            LTMGPT2Block(self.base_model.h[-cnt_blocks_with_memory+i]) for i in range(cnt_blocks_with_memory)
        ])
        self.ln_f = self.base_model.ln_f

        # Model parallel
        self.model_parallel = self.base_model.model_parallel
        self.device_map = self.base_model.device_map
        self.gradient_checkpointing = self.base_model.gradient_checkpointing
        
        # Initialize weights and apply final processing
        # self.post_init()
        
    
    
    @add_start_docstrings_to_model_forward(GPT2_INPUTS_DOCSTRING)
    @add_code_sample_docstrings(
        checkpoint=_CHECKPOINT_FOR_DOC,
        output_type=BaseModelOutputWithPastAndCrossAttentions,
        config_class=_CONFIG_FOR_DOC,
    )
    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[Tuple[Tuple[torch.Tensor]]] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.Tensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, BaseModelOutputWithPastAndCrossAttentions]:
        output_attentions = output_attentions if output_attentions is not None else self.base_model.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.base_model.config.output_hidden_states
        )
        use_cache = use_cache if use_cache is not None else self.base_model.config.use_cache
        return_dict = return_dict if return_dict is not None else self.base_model.config.use_return_dict

        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            self.base_model.warn_if_padding_and_no_attention_mask(input_ids, attention_mask)
            input_shape = input_ids.size()
            input_ids = input_ids.view(-1, input_shape[-1])
            batch_size = input_ids.shape[0]
        elif inputs_embeds is not None:
            input_shape = inputs_embeds.size()[:-1]
            batch_size = inputs_embeds.shape[0]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        device = input_ids.device if input_ids is not None else inputs_embeds.device

        if token_type_ids is not None:
            token_type_ids = token_type_ids.view(-1, input_shape[-1])

        if past_key_values is None:
            past_length = 0
            past_key_values = tuple([None] * len(self.h))
        else:
            past_length = past_key_values[0][0].size(-2)
        if position_ids is None:
            position_ids = torch.arange(past_length, input_shape[-1] + past_length, dtype=torch.long, device=device)
            position_ids = position_ids.unsqueeze(0)

        # GPT2Attention mask.
        if attention_mask is not None:
            if batch_size <= 0:
                raise ValueError("batch_size has to be defined and > 0")
            attention_mask = attention_mask.view(batch_size, -1)
            # We create a 3D attention mask from a 2D tensor mask.
            # Sizes are [batch_size, 1, 1, to_seq_length]
            # So we can broadcast to [batch_size, num_heads, from_seq_length, to_seq_length]
            # this attention mask is more simple than the triangular masking of causal attention
            # used in OpenAI GPT, we just need to prepare the broadcast dimension here.
            attention_mask = attention_mask[:, None, None, :]

            # Since attention_mask is 1.0 for positions we want to attend and 0.0 for
            # masked positions, this operation will create a tensor which is 0.0 for
            # positions we want to attend and the dtype's smallest value for masked positions.
            # Since we are adding it to the raw scores before the softmax, this is
            # effectively the same as removing these entirely.
            attention_mask = attention_mask.to(dtype=self.dtype)  # fp16 compatibility
            attention_mask = (1.0 - attention_mask) * torch.finfo(self.dtype).min

        # If a 2D or 3D attention mask is provided for the cross-attention
        # we need to make broadcastable to [batch_size, num_heads, seq_length, seq_length]
        if self.base_model.config.add_cross_attention and encoder_hidden_states is not None:
            encoder_batch_size, encoder_sequence_length, _ = encoder_hidden_states.size()
            encoder_hidden_shape = (encoder_batch_size, encoder_sequence_length)
            if encoder_attention_mask is None:
                encoder_attention_mask = torch.ones(encoder_hidden_shape, device=device)
            encoder_attention_mask = self.invert_attention_mask(encoder_attention_mask) # self.base_model.
        else:
            encoder_attention_mask = None

        # Prepare head mask if needed
        # 1.0 in head_mask indicate we keep the head
        # attention_probs has shape bsz x n_heads x N x N
        # head_mask has shape n_layer x batch x n_heads x N x N
        head_mask = self.get_head_mask(head_mask, self.base_model.config.n_layer) # self.base_model.

        if inputs_embeds is None:
            inputs_embeds = self.wte(input_ids)
        position_embeds = self.wpe(position_ids)
        hidden_states = inputs_embeds + position_embeds

        if token_type_ids is not None:
            token_type_embeds = self.wte(token_type_ids)
            hidden_states = hidden_states + token_type_embeds

        hidden_states = self.drop(hidden_states)

        output_shape = (-1,) + input_shape[1:] + (hidden_states.size(-1),)

        if self.gradient_checkpointing and self.training:
            if use_cache:
                logger.warning_once(
                    "`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`..."
                )
                use_cache = False

        presents = () if use_cache else None
        all_self_attentions = () if output_attentions else None
        all_cross_attentions = () if output_attentions and self.config.add_cross_attention else None
        all_hidden_states = () if output_hidden_states else None
        for i, (block, layer_past) in enumerate(zip(self.h, past_key_values)):
            # Model parallel
            if self.model_parallel:
                torch.cuda.set_device(hidden_states.device)
                # Ensure layer_past is on same device as hidden_states (might not be correct)
                if layer_past is not None:
                    layer_past = tuple(past_state.to(hidden_states.device) for past_state in layer_past)
                # Ensure that attention_mask is always on the same device as hidden_states
                if attention_mask is not None:
                    attention_mask = attention_mask.to(hidden_states.device)
                if isinstance(head_mask, torch.Tensor):
                    head_mask = head_mask.to(hidden_states.device)
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            if self.gradient_checkpointing and self.training:
                outputs = self._gradient_checkpointing_func(
                    block.__call__,
                    hidden_states,
                    None,
                    attention_mask,
                    head_mask[i],
                    encoder_hidden_states,
                    encoder_attention_mask,
                    use_cache,
                    output_attentions,
                )
            else:
                outputs = block(
                    hidden_states,
                    layer_past=layer_past,
                    attention_mask=attention_mask,
                    head_mask=head_mask[i],
                    encoder_hidden_states=encoder_hidden_states,
                    encoder_attention_mask=encoder_attention_mask,
                    use_cache=use_cache,
                    output_attentions=output_attentions,
                )

            hidden_states = outputs[0]
            if use_cache is True:
                presents = presents + (outputs[1],)

            if output_attentions:
                all_self_attentions = all_self_attentions + (outputs[2 if use_cache else 1],)
                if self.config.add_cross_attention:
                    all_cross_attentions = all_cross_attentions + (outputs[3 if use_cache else 2],)

            # Model Parallel: If it's the last layer for that device, put things on the next device
            if self.model_parallel:
                for k, v in self.device_map.items():
                    if i == v[-1] and "cuda:" + str(k) != self.last_device:
                        hidden_states = hidden_states.to("cuda:" + str(k + 1))

        memory = hidden_states
        
        for i, block in enumerate(self.transformer_ltm_blocks_h): # TODO add flags like in `for` up
            block.update_memory(memory)
            hidden_states = block(hidden_states)
        
        hidden_states = self.ln_f(hidden_states)

        hidden_states = hidden_states.view(output_shape)
        # Add last hidden state
        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        if not return_dict:
            return tuple(
                v
                for v in [hidden_states, presents, all_hidden_states, all_self_attentions, all_cross_attentions]
                if v is not None
            )

        return BaseModelOutputWithPastAndCrossAttentions(
            last_hidden_state=hidden_states,
            past_key_values=presents,
            hidden_states=all_hidden_states,
            attentions=all_self_attentions,
            cross_attentions=all_cross_attentions,
        )    

In [ ]:
model.transformer.config

In [ ]:
model

In [ ]:
model

In [ ]:
model.transformer = LTM_GPT2Model(model.transformer)
model

In [ ]:
LTM_GPT2Model

In [ ]:
# class LTM_GPT(GPT2LMHeadModel):
#     """ Custom LTM GPT2 layer with memory """
#     def __init__(self, model: GPT2LMHeadModel, cnt_blocks_with_memory=2):
#         super().__init__()
#         self.base_model = model
#         self.transformer = self.base_model.tranformer GPT2Model(config)
#         self.transformer.h = self.base_model.transformer.h[:-cnt_blocks_with_memory]
        
#         self.transformer_ltm_blocks = nn.ModuleList([
#             LTMGPT2Block(self.base_model.transformer.h[-cnt_blocks_with_memory+i]) for i in range(cnt_blocks_with_memory)
#         ])
        
#         self.lm_head = self.base_model.lm_head

#         # Model parallel
#         # self.model_parallel = False
#         # self.device_map = None

#         # Initialize weights and apply final processing
#         # self.post_init()
    
#     @add_start_docstrings_to_model_forward(GPT2_INPUTS_DOCSTRING)
#     @add_code_sample_docstrings(
#         checkpoint=_CHECKPOINT_FOR_DOC,
#         output_type=CausalLMOutputWithCrossAttentions,
#         config_class=_CONFIG_FOR_DOC,
#     )
#     def forward(
#         self,
#         input_ids: Optional[torch.LongTensor] = None,
#         past_key_values: Optional[Tuple[Tuple[torch.Tensor]]] = None,
#         attention_mask: Optional[torch.FloatTensor] = None,
#         token_type_ids: Optional[torch.LongTensor] = None,
#         position_ids: Optional[torch.LongTensor] = None,
#         head_mask: Optional[torch.FloatTensor] = None,
#         inputs_embeds: Optional[torch.FloatTensor] = None,
#         encoder_hidden_states: Optional[torch.Tensor] = None,
#         encoder_attention_mask: Optional[torch.FloatTensor] = None,
#         labels: Optional[torch.LongTensor] = None,
#         use_cache: Optional[bool] = None,
#         output_attentions: Optional[bool] = None,
#         output_hidden_states: Optional[bool] = None,
#         return_dict: Optional[bool] = None,
#     ) -> Union[Tuple, CausalLMOutputWithCrossAttentions]:
#         r"""
#         labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
#             Labels for language modeling. Note that the labels **are shifted** inside the model, i.e. you can set
#             `labels = input_ids` Indices are selected in `[-100, 0, ..., config.vocab_size]` All labels set to `-100`
#             are ignored (masked), the loss is only computed for labels in `[0, ..., config.vocab_size]`
#         """
#         return_dict = return_dict if return_dict is not None else self.base_model.config.use_return_dict

#         transformer_outputs = self.transformer(
#             input_ids,
#             past_key_values=past_key_values,
#             attention_mask=attention_mask,
#             token_type_ids=token_type_ids,
#             position_ids=position_ids,
#             head_mask=head_mask,
#             inputs_embeds=inputs_embeds,
#             encoder_hidden_states=encoder_hidden_states,
#             encoder_attention_mask=encoder_attention_mask,
#             use_cache=use_cache,
#             output_attentions=output_attentions,
#             output_hidden_states=output_hidden_states,
#             return_dict=return_dict,
#         )
#         hidden_states = transformer_outputs[0]
        
#         # Init memory as hidden_states from 37 layers
#         hidden_states

#         # Set device for model parallelism
#         if self.base_model.model_parallel:
#             torch.cuda.set_device(self.transformer.first_device)
#             hidden_states = hidden_states.to(self.lm_head.weight.device)

#         lm_logits = self.lm_head(hidden_states)

#         loss = None
#         if labels is not None:
#             # move labels to correct device to enable model parallelism
#             labels = labels.to(lm_logits.device)
#             # Shift so that tokens < n predict n
#             shift_logits = lm_logits[..., :-1, :].contiguous()
#             shift_labels = labels[..., 1:].contiguous()
#             # Flatten the tokens
#             loss_fct = CrossEntropyLoss()
#             loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

#         if not return_dict:
#             output = (lm_logits,) + transformer_outputs[1:]
#             return ((loss,) + output) if loss is not None else output

#         return CausalLMOutputWithCrossAttentions(
#             loss=loss,
#             logits=lm_logits,
#             past_key_values=transformer_outputs.past_key_values,
#             hidden_states=transformer_outputs.hidden_states,
#             attentions=transformer_outputs.attentions,
#             cross_attentions=transformer_outputs.cross_attentions,
#         )

In [ ]:
model

In [ ]:
model

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad) / sum(p.numel() for p in model.parameters())
count_parameters(model)

In [ ]:
for param in model.parameters():
    param.requires_grad = False

print(count_parameters(model))

In [ ]:
for param in model.parameters():
    param.requires_grad = False

print(count_parameters(model))

for param in model.transformer.transformer_ltm_blocks_h.parameters():
    param.requires_grad=True
    # param.data = param.data.to(torch.float32)

for param in model.transformer.ln_f.parameters():
    param.requires_grad=True
    # param.data = param.data.to(torch.float32)

for param in model.lm_head.parameters():
    param.requires_grad=True
    # param.data = param.data.to(torch.float32)

print(count_parameters(model))

In [ ]:
# # Init LTM gpt blocks
# model.transformer.h[-2] = LTMGPT2Block(model.transformer.h[-2])
# model.transformer.h[-1] = LTMGPT2Block(model.transformer.h[-1])

In [ ]:
# # Upcast
# for param in model.transformer.h[-2:].parameters():
#     param.data = param.data.to(torch.float32)
    
# for param in model.transformer.ln_f.parameters():
#     param.requires_grad=True
#     param.data = param.data.to(torch.float32)

# for param in model.lm_head.parameters():
#     param.requires_grad=True
#     param.data = param.data.to(torch.float32)

In [ ]:
print(count_parameters(model))

In [ ]:
model = model.to(device)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruGPT-3.5-13B")

In [ ]:
code_dataset = load_dataset("codeparrot/codeparrot-clean-valid")

In [ ]:
prompts =  ['import', 'from', 'while', 'try', 'if', 'for', 'torch']  # feel free to add a few more that are not 100% assiciated with Python

MAX_STEPS = 100

for prompt in tqdm(prompts):
    print(tokenizer(prompt, return_tensors='pt', return_token_type_ids=False))

In [ ]:
def custom_generate(prompt, model, device, max_steps):
    batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
    print(batch)

    for i in range(max_steps):
        outputs = model(**batch)
        #print(outputs)
        probs = outputs.logits[0, -1].nan_to_num(nan=0.0).div(0.8).softmax(-1) #.argmax(-1).reshape(1, 1)
        old_token = outputs.logits[0, -1].argmax(-1).reshape(1, 1)
        #print(old_token)
        next_token = torch.multinomial(probs, 1).reshape(1, 1)
        #print(next_token)
        batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
        batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)
        break

    return tokenizer.decode(batch['input_ids'][0].cpu().numpy().tolist()[1:])

In [ ]:
model

In [ ]:
after_finetuning_samples = []
for prompt in tqdm(prompts):
    after_finetuning_samples.append(custom_generate(prompt, model, device, MAX_STEPS))
after_finetuning_samples